In [ ]:
import os
import json
os.chdir('/mnt/openfact/users/msawinski/factue-task2')
from langchain.prompts import ChatPromptTemplate
from factue.methods.llm_langchain.llm import Llm

In [2]:
chat_model = Llm(
            model_name='llama3.1:8b',# type: ignore
            provider='ollama',  # type: ignore
            mode='chat',  # type: ignore
            temperature='0.0',  # type: ignore
            seed=42
        )

seed:  42


In [ ]:
def improve_claim(claim, post, prmpt_system, prmpt_user, promt_format, chat_model):
    prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", prmpt_system+promt_format),
        ("human", prmpt_user),
    ]
)
    print(claim)
    while claim and len(claim) > 0:
        prompt = prompt_template.invoke(
        {
            "post": post,
            "claim": claim
        })
        reposonse = chat_model.invoke(prompt).content

        try:
            response = json.loads(reposonse)
            rating = response['rating']
            reason = response['reason']
            alternative = response['improved_claim']
            print(str(rating)+" | "+reason)
            print("*"*200)

            print(alternative)
        except:
            print('Error in response')
            print(reposonse)
            claim = None
        if alternative and claim != alternative:
            claim = alternative
        else:
            print("*"*200)
            print("*"*200)
            break

In [4]:
claim = """Photo shows Louis Armstrong as a child"""
post = """The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional"""

prmpt_system = """You are a “Normalized Claim Generator.” 
Input: a noisy, unstructured social-media post.
Output:
- Include only information from the original post; minimal disambiguation allowed.
- Use formal, simple, non-offensive language.
- Preserve all named entities exactly as in the input.
- Keep numerical values intact; convert units when helpful.
- Retain original wording where possible; avoid fluff.
- Preserve semantic meaning; do not correct factual errors.
- Maintain a neutral, unbiased tone; no moral judgment.
- Expand acronyms and shorthand (e.g., “NYC” → “New York City”) unless part of an official name.

You will receive a claim generated before and an orginal post.
Your task is to rate the claim on a scale from 0 to 10, where 0 means the claim is completely wrong and 10 means the claim is completely correct. Generate reason for the rating.
If the claim is not correct, you need to formulate a new one sentence claim that better represents the text of post.
"""

promt_format = """\nOutput ONLY JSON: in the format {{"rating": 0-10, "reason": "reason for rating","alternative": "formulate new one sentence claim that better represents the text of post}}."""

prmpt_user = """POST:\n{post}\n\nCLAIM:\n{claim}"""

improve_claim(claim, post, prmpt_system, prmpt_user, promt_format, chat_model)

Photo shows Louis Armstrong as a child
8 | The claim is mostly correct, but it doesn't mention the Karnofsky family's name or their connection to the boy. The post also implies that the boy was employed by the Karnofskys and received financial support from them.
********************************************************************************************************************************************************************************************************
Photo shows Louis Armstrong as a child who was employed by the Karnofsky Jewish family in the United States.
8 | The claim is mostly correct, but it lacks specific details about the employment and adoption of the boy. The post mentions that the boy was given homework to get food, which implies a more complex situation than just being employed.
**********************************************************************************************************************************************************************************************

In [14]:
claim = """Photo shows Louis Armstrong as a child"""
post = """The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional"""

prmpt_system = """You are a “Normalized Claim Generator.”  
Input: a noisy, unstructured social-media post plus a previously generated claim.  
Task:
1. Rate the claim’s faithfulness on a 0–10 scale (0 = completely wrong, 10 = fully correct).  
2. Explain your rating in a brief reason.  
3. If the rating is less than 10, rewrite a single‐sentence claim that better reflects the post.  

When generating or evaluating a claim, it must:
- Include **only** information present in the original post (minimal disambiguation allowed).
- **Not** introduce or infer any new entities, identities, or facts not explicitly mentioned in post. 
- Use clear, formal, simple, non-offensive language.  
- Preserve every named entity exactly as in the input.  
- Keep numerical values intact; convert units only when necessary.  
- Retain original phrasing where possible; remove all filler.  
- Preserve the post’s semantic meaning; do not correct factual errors.  
- Maintain a neutral, unbiased tone without moral judgment.  
- Expand acronyms and shorthand (e.g., “NYC” → “New York City”) unless part of an official name.
- If the post contains too much information for one sentence, select only the most important claim."""

promt_format = """
Output **only** valid JSON, for example:
{{
  "rating": 7,
  "reason": "The claim captures the main event but omits the time and location mentioned in the post.",
  "improved_claim": "John Doe completed the 5K race in Central Park on August 12 in 22 minutes."
}}"""

prmpt_user = """POST:\n{post}\n\nCLAIM:\n{claim}"""

improve_claim(claim, post, prmpt_system, prmpt_user, promt_format, chat_model)






Photo shows Louis Armstrong as a child
2 | The claim is partially correct but does not accurately describe the post's content. The post mentions a Jewish family and a boy who was employed by them, whereas the claim refers to a photo of Louis Armstrong.
********************************************************************************************************************************************************************************************************
A young boy from a Jewish family in Lithuania was employed by the Karnofsky family and showed musical talent.
8 | The claim captures the essence of the post but omits some specific details such as the boy's age, his initial purpose for working with the Karnofsky family (to get food), and the fact that he was adopted into their home.
********************************************************************************************************************************************************************************************************
A 7-year-o

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", prmpt_system+promt_format),
        ("human", prmpt_user),
    ]
)
prompt = prompt_template.invoke(
        {
            "post": post,
            "claim": claim
        })
prompt
print("\n".join([p.content for p in prompt.messages]))

You are a “Normalized Claim Generator.”  
Input: a noisy, unstructured social-media post plus a previously generated claim.  
Task:
1. Rate the claim’s faithfulness on a 0–10 scale (0 = completely wrong, 10 = fully correct).  
2. Explain your rating in a brief reason.  
3. If the rating is less than 10, rewrite a single‐sentence claim that better reflects the post.  

When generating or evaluating a claim, it must:
- Include **only** information present in the original post (minimal disambiguation allowed).
- **Not** introduce or infer any new entities, identities, or facts not explicitly mentioned in post. 
- Use clear, formal, simple, non-offensive language.  
- Preserve every named entity exactly as in the input.  
- Keep numerical values intact; convert units only when necessary.  
- Retain original phrasing where possible; remove all filler.  
- Preserve the post’s semantic meaning; do not correct factual errors.  
- Maintain a neutral, unbiased tone without moral judgment.  
- Ex

In [7]:
prompt.messages()

TypeError: 'list' object is not callable